In [ ]:
import os
import openai
import json
import gradio as gr
from openai import OpenAI
from pathlib import Path

# Pour GoogleColab
# from google.colab import userdata 
# Hors GoogleColab 3 lignes ci-dessous
from dotenv import load_dotenv
import os
load_dotenv(override=True)


# Ticket price function
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for city {destination_city}")
    price = ticket_prices.get(destination_city.lower(), "Unknown ticket price")
    return f"The price of a ticket to {destination_city} is {price}"

# Weather function
city_weather = {"london": "21°", "paris": "22°", "tokyo": "15°", "berlin": "14°"}

def get_weather(destination_city):
    print(f"Tool get_weather called for city {destination_city}")
    temperature = city_weather.get(destination_city.lower(), "Unknown city")
    return f"The temperature in {destination_city} is {temperature}"

# This dictionary structure is required to describe the get_ticket_price function as a tool
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

# This dictionary structure is required to describe the get_weather function as a tool
weather_function = {
    "name": "get_weather",
    "description": "Get the temperature of a given city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants the weather",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

# This is the list of our tools
tools = [{"type": "function", "function": price_function},
        {"type": "function", "function": weather_function}]


class Chat:

    def __init__(self):
        #self.openai_api_key = userdata.get('OPENAI_API_KEY') # Pour GoogleColab
        self.openai_api_key = os.getenv('OPENAI_API_KEY')     # Hors GoogleColab 
        self.client = OpenAI(api_key=self.openai_api_key)

        
        

    def handle_tool_call(self, tool_calls):
        results = []
        for tool_call in tool_calls:
            tool_name = tool_call.function.name
            arguments = json.loads(tool_call.function.arguments)
            print(f"Tool called: {tool_name} with args: {arguments}", flush=True)
            tool = globals().get(tool_name)
            result = tool(**arguments) if tool else {}
            results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
        return results

    def system_prompt(self):
        system_prompt = """
You are a helpful assistant for an Airline called FlightAI.
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so.
"""
        return system_prompt

    def chat(self, message, history):
        messages = [{"role": "system", "content": self.system_prompt()}] + history + [{"role": "user", "content": message}]
        done = False
        while not done:
            response = self.client.chat.completions.create(model="gpt-4o-mini-2024-07-18", messages=messages, tools=tools)
            if response.choices[0].finish_reason=="tool_calls":
                message = response.choices[0].message
                tool_calls = message.tool_calls
                results = self.handle_tool_call(tool_calls)
                messages.append(message)
                messages.extend(results)
            else:
                done = True
        return response.choices[0].message.content




/home/matos/projects/tests_ia_agentique/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
chat = Chat()
#gr.ChatInterface(chat.chat, type="messages").launch(debug=True) # Pour GoogleColab
gr.ChatInterface(chat.chat).launch(debug=True)                   # Hors GoogleColab    